In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
# 1) 케라스 오토인코더 로드
autoencoder = load_model("autoencoder.h5")
print("autoencoder 로드 완료.")

In [ ]:
# 2) HMM 모델들 로드
gesture_models = {}
save_dir = "saved_hmm_models"
for file_name in os.listdir(save_dir):
    if file_name.endswith(".pkl"):
        gesture_name = file_name.replace("hmm_", "").replace(".pkl", "")
        model_path = os.path.join(save_dir, file_name)
        hmm_model = joblib.load(model_path)
        gesture_models[gesture_name] = hmm_model
print(f"HMM 모델 {len(gesture_models)}개 로드 완료:", list(gesture_models.keys()))

In [ ]:
# 3) Scaler 로드 (훈련 때 사용한 scaler.pkl 가 있다고 가정)
scaler = joblib.load("scaler.pkl")
print("Scaler 로드 완료.")

In [ ]:
# 4) Mediapipe 세팅
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

encoder = load_model("encoder.h5")  # encoder만 로드했다고 가정
print("encoder 로드 완료.")

In [ ]:
def classify_hmm(latent_seq_2d, gesture_models):
    """
    latent_seq_2d: shape=(T, latent_dim)
    gesture_models: { gesture_name: hmm_model, ... }
    return: (best_label, best_score)
    """
    best_label = None
    best_score = -np.inf
    length = [latent_seq_2d.shape[0]]
    for gesture_name, model in gesture_models.items():
        score = model.score(latent_seq_2d, length)
        if score > best_score:
            best_score = score
            best_label = gesture_name
    return best_label, best_score

def get_landmark_63(hand_landmarks):
    """
    Mediapipe의 hand_landmarks(21개)에 대해 (x,y,z) 63차원 리스트 반환
    """
    row_data = []
    for lm in hand_landmarks.landmark:  # 21개
        row_data.extend([lm.x, lm.y, lm.z])
    return row_data

import time

# 웹캠 열기
cap = cv2.VideoCapture(0)  # 0번 카메라

FRAME_WINDOW = 60  # 누적 프레임 수 (예시: 60프레임=약2초)
sequence_data = []

while True:
    success, frame = cap.read()
    if not success:
        print("카메라에서 영상을 받지 못했습니다.")
        break

    # 이미지 좌우 반전(편의상)
    frame = cv2.flip(frame, 1)

    # Mediapipe 입력을 위해 BGR->RGB 변환
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    # 현재 프레임의 (x,y,z) 63차원 추출
    if results.multi_hand_landmarks:
        # 하나의 손만 사용한다고 가정
        hand_landmarks = results.multi_hand_landmarks[0]
        row_data = get_landmark_63(hand_landmarks)
    else:
        # 손이 검출 안 되었으면 63차원 0
        row_data = [0.0]*63

    sequence_data.append(row_data)

    # 일정 길이 쌓이면 한 번 분류
    if len(sequence_data) >= FRAME_WINDOW:
        # numpy 변환 (T, 63)
        seq_array = np.array(sequence_data)

        # 1) Scaler 적용 -> (T, 63)
        seq_scaled = scaler.transform(seq_array)

        # 2) 차원 맞추기 => (1, T, 63)
        seq_scaled = np.expand_dims(seq_scaled, axis=0)

        # 3) 인코더 통과 => (1, T, latent_dim)
        latent_seq = encoder.predict(seq_scaled)  
        # latent_seq.shape = (1, T, latent_dim)

        # hmm은 (T, latent_dim) 필요
        latent_seq_2d = latent_seq[0]

        # 4) HMM 분류
        pred_label, pred_score = classify_hmm(latent_seq_2d, gesture_models)
        print(f"[HMM] Predicted gesture: {pred_label} (score={pred_score:.2f})")

        # 결과 표시(화면에 텍스트로)
        cv2.putText(frame, f"Pred: {pred_label}", (50,50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)

        # 다음번 인식을 위해 sequence_data 초기화
        # (혹은 슬라이딩 윈도우로 일부 프레임만 유지하도록 설계 가능)
        sequence_data = []

    # 디스플레이
    cv2.imshow("Real-time Hand Gesture", frame)

    # ESC 키(27) 누르면 종료
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()